In [8]:
import math
import random
import numpy as np

In [9]:
# Lmax: Retransmit limit
# TXOP: in ms
# Imax: total slots in worst case
def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, mode='general'):
    total_power = 0
    counter_delay_slot=0
    success_sta=0
    # setting ocw_n [ocw_min, 1st_retry_ocw....]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time
    retrans_counter = [0 for _ in range(0, sta_amount)]
#     # selected RA-RU index for each station
#     sta_selected_R = [None for _ in range(0, sta_amount)]
    # ------------------------------------ STA setting ------------------------------------------
    # number of STAs on RA-RU
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    
    success_info = []
    # No.i slot    
    for i in range(0, Imax):
        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < 5):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # Record power (no power control so always 1)
            total_power += len(sta_on_R[i][r]) * 1
            
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record
                counter_delay_slot += (i+1)
                success_sta += 1
                # no power control so always 1
#                 total_power += 1 * (retrans_counter[success_sta_index] + 1)
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < 5):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])
#     print(counter_delay_slot)
#     print(success_sta)
    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    average_power = 0
    if success_sta > 0:
        average_power = total_power / success_sta
#     average_power = total_power / sta_amount
    
    return success_probability, access_delay, through_put, utilization, average_power

In [10]:
sample = 1000

# 1, 2, ... 10, 20, 30, ... 100, 200, 300, ... 1000
m_list_simu = np.concatenate([np.arange(1, 11, 1), np.arange(20, 110, 10), np.arange(200, 1100, 100)])
Ps_list_simu = [0 for i in range(28)]
Da_list_simu = [0 for i in range(28)]
T_list_simu = [0 for i in range(28)]
U_list_simu = [0 for i in range(28)]
power_list_simu = [0 for i in range(28)]

# m_list_simu = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
# Ps_list_simu = [0 for i in range(20)]
# Da_list_simu = [0 for i in range(20)]
# T_list_simu = [0 for i in range(20)]
# U_list_simu = [0 for i in range(20)]

Lmax=5
TXOP=5.673
ocw_min=7
ocw_max=31

R_list = [9,18,37,74,148]
for R in R_list:
    for idx, m in enumerate(m_list_simu):
        success_probability_total = 0;
        access_delay_total = 0;
        through_put_total = 0;
        utilization_total = 0;
        power_usage_total = 0;
        no_success_counter = 0;
        for i in range(sample):
            success_probability, access_delay, through_put, utilization, power_usage = simulation(sta_amount=m, RA_RU=R, Lmax=Lmax, TXOP=TXOP, ocw_min=ocw_min, ocw_max=ocw_max, mode='general')
            success_probability_total += success_probability
            access_delay_total += access_delay
            through_put_total += through_put
            utilization_total += utilization
            power_usage_total += power_usage

        Ps = success_probability_total / sample
        Da = access_delay_total / sample
        T = through_put_total / sample
        U = utilization_total / sample
#         power = power_usage_total / (sample - no_success_counter)
        if (sample - no_success_counter) > 0:
            power = power_usage_total / (sample - no_success_counter)

        Ps_list_simu[idx] = Ps
        Da_list_simu[idx] = Da
        T_list_simu[idx] = T
        U_list_simu[idx] = U
        power_list_simu[idx] = power

    print(f'ps_list_simu_{R} = {Ps_list_simu}')
    print(f'da_list_simu_{R} = {Da_list_simu}')
    print(f't_list_simu_{R} = {T_list_simu}')
    print(f'u_list_simu_{R} = {U_list_simu}')
    print(f'power_list_simu_{R} = {power_list_simu} \n')

ps_list_simu_9 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9997999999999999, 0.9907000000000015, 0.9026666666666622, 0.7092500000000005, 0.5145800000000003, 0.37571666666666687, 0.2789285714285714, 0.21583749999999988, 0.16768888888888914, 0.13787999999999997, 0.04509499999999981, 0.025990000000000058, 0.018182499999999973, 0.013307999999999915, 0.010820000000000005, 0.008759999999999987, 0.007312499999999995, 0.006397777777777762, 0.005415999999999951]
da_list_simu_9 = [5.672999999999856, 6.696976499999851, 7.410828999999852, 8.057078249999888, 8.812438199999981, 9.906949, 10.925387571428525, 11.881389375000047, 12.517159333333293, 13.54258560000006, 23.73256768320439, 32.81551105358449, 39.24668388758677, 43.74820281429042, 47.562319458933366, 51.032473824096265, 54.140734905131175, 57.1298891142195, 59.304765028487246, 69.66799627856699, 72.5954199926992, 74.1330774332835, 75.32684205642703, 76.22168509140042, 76.70414479077618, 77.19221574771092, 77.59827048311729, 78.1168641

ps_list_simu_148 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9955300000000031, 0.8035466666666659, 0.4298099999999999, 0.2033899999999989, 0.09729666666666681, 0.04700714285714272, 0.023227499999999957, 0.011787777777777756, 0.005794999999999954]
da_list_simu_148 = [5.672999999999856, 5.712710999999856, 5.759985999999858, 5.782205249999861, 5.82957479999987, 5.823334499999872, 5.9153181428570045, 5.970123374999907, 5.956019666666558, 6.015081899999913, 6.384110550000015, 6.700947600000007, 7.088129850000018, 7.418128260000028, 7.791014550000005, 8.166445585714301, 8.508578137500036, 8.870933133333338, 9.329135040000002, 14.042745440398424, 17.857121723019524, 18.390666271752412, 18.037540930091488, 17.642934747579837, 17.39780005029307, 17.33293247673548, 17.138792521534377, 16.90111083783717]
t_list_simu_148 = [0.19999999999999718, 0.39999999999999436, 0.6000000000000113, 0.7999999999999887, 1.0, 1.2000000000000226, 1.40000000000